# 17. SAC 算法
> 尽管 **DDPG** 属于 **离线策略算法** ，具有一定的样本效率。但是它的训练不稳定，收敛性较差，需要细致的超参数调整，难以适应复杂环境。
> 2018 年，Tuomas 等人提出了一个更加稳定的 **离线策略算法**： [Soft Actor-Critic: Off-Policy Maximum Entropy Deep Reinforcement Learning with a Stochastic Actor](https://arxiv.org/abs/1801.01290)
> **SAC** 的前身是 **Soft Q-learning**，它们都属于 **最大熵** 强化学习的范畴。**Soft Q-learning** 不存在一个 **显式的策略函数**，而是使用一个 **Q函数的最大熵（Boltzmann）分布**：
$$\pi(a|s)=\frac{e^{\frac{Q(s,a)}{\alpha}}}{\sum_{a^{\prime}}e^{\frac{Q(s,a^{\prime})}{\alpha}}}$$
- 当$\alpha$较大时,$\frac{Q(s,a)}{\alpha}$的值变小,导致所有的$\exp$值接近 1，所有动作的概率大致相等，策略变得更为均匀，鼓励 **更多的探索**
- 当$\alpha$较小时,$\frac{Q(s,a)}{\alpha}$的值变大,导致所有的$\exp$值远大于其他动作，所有动作的概率差异变大，倾向于 **更多的利用**
- 这种方法在离散动作空间下比较容易实现，因为可以直接枚举动作求分布。但在 连续动作空间 下，计算归一化因子（积分）非常困难，近似计算代价很大。于是 SAC 提出使用一个 **Actor 表示策略函数**，从而解决这个问题。
> 目前，在**无模型**的强化学习算法中，**SAC** 是一个非常高效的算法，它学习一个 **随机性策略**，在不少标准环境中取得了领先的成绩。

## 17.1 SAC算法原理
- 最大熵强化学习
- Soft 策略迭代

### 最大熵强化学习
> **熵（Entropy）** 本质上是**不确定性**或**信息量的期望**的度量。它最早由 **香农在信息论** 中提出，用来描述一个随机变量的 **不确定程度**
> 如果一个离散随机变量 $X$ 的概率分布是：
$$
P(X = x_i) = p_i,\quad i=1,\dots, n
$$
> 那么它的**信息熵**定义为：
$$
H(X) = - \sum_{i=1}^n p_i \log p_i
$$

- $p_i$ 越均匀，熵越大，表示越不确定
- $p_i$ 越集中（某个事件几乎必然发生），熵越小，甚至为 $0$（完全确定）
> **例子**：
- 公平硬币：$p(\text{正})=p(\text{反})=0.5$，熵最大 $H=1$ （最不确定）
- 偏置硬币：$p(\text{正})=0.9$，熵小于 $1$（更确定）
- 完全确定事件：$p=1$ 或 $p=0$，熵为 $0$（无不确定性）

> 如果是连续随机变量 $X$，熵的定义为**微分熵**：
$$
H(X) = -\int p(x) \log p(x) \, dx
$$
---
> 传统强化学习的目标是在一条轨迹$\tau$下 **最大化期望累积回报**：
$$
J(\pi) = \mathbb{E}_{\tau\sim\pi} \left[ \sum_{t=0}^{\infty} \gamma^t r(s_t,a_t) \right]
$$
- 目标**只关心奖励**，并不会**刻意**鼓励探索，可能会早早陷入次优策略
---
> 而 **最大熵强化学习** 把策略的 **不确定性（熵）** 也纳入优化目标：
$$
J(\pi) = \mathbb{E}_{\tau\sim\pi} \left[ \sum_{t=0}^{\infty} \gamma^t \big( r(s_t,a_t) + \alpha \, \mathcal{H}(\pi(\cdot|s_t)) \big) \right]
$$

- $\mathcal{H}(\pi(\cdot|s_t)) = - \sum_a \pi(a|s_t) \log \pi(a|s_t)$ 表示在状态 $s_t$ 下动作分布的熵
- $\alpha$ 也是**温度参数**，权衡**奖励最大化**与**探索性**：
  - $\alpha$ 大 → 更注重探索（熵高）
  - $\alpha$ 小 → 更注重利用（奖励高）
  - 有助于加速后续的策略学习，并减少策略陷入较差的局部最优的可能性


![传统强化学习和最大熵强化学习的区别](Illustrations/传统强化学习和最大熵强化学习的区别.png)

### Soft 策略迭代